# Imports

In [2]:
import requests
import bs4
import json
import datetime as dt
import sys
import pandas as pd
import geopy
import geopandas as gpd
import re
import warnings
from shapely import wkt
from shapely.geometry import Point
import math
warnings.filterwarnings('ignore')

# Lisbon Properties for Sale - SUPERCASA Webscrapping

In [181]:
num_prop_per_page = 25
total_pages = 75
time_estimation = '3 to 5'

print(f'Time estimation for this task: {time_estimation} minutes.')
estimation = (dt.datetime.now() + dt.timedelta(minutes=4))
start = f'Tasks started at {dt.datetime.now().hour}h:{dt.datetime.now().minute}min. Estimated finish time {estimation.hour}h:{estimation.minute}min.'
print(start)

title, price, num_rooms, total_area, latitude, longitude, region, extras, id = ([] for i in range(9))

for n in range(1, 75):
    url = f'https://supercasa.pt/comprar-casas/lisboa/pagina-{n}'
    result = requests.get(url)
    soup = bs4.BeautifulSoup(result.text, 'lxml')

    # Find all properties on the current page
    properties = soup.find_all('div', class_='property big-picture') 

    for prop in properties:
        # Title
        a = prop.find('h2', class_='property-list-title').find('a')
        title.append(a.get_text().strip() if a else ' ')

        # Price
        span = prop.find('div', class_='property-price').find('span')
        price.append(span.get_text(strip=True) if span else ' ')

        # Features
        feature = prop.find('div', class_='property-features')
        spans = feature.find_all('span') if feature else []
        rooms = spans[0].get_text() if len(spans) > 0 else "Unknown"
        area = spans[1].get_text() if len(spans) > 1 else "Unknown"
        num_rooms.append(rooms)
        total_area.append(area)

        # Links for latitude and longitude
        link = prop.find('a', class_='property-link')
        latitude.append(link.get('data-latitude') if link else 'Unknown')
        longitude.append(link.get('data-longitude') if link else 'Unknown')

        # Extras
        highlight = prop.find('div', class_='property-highlights')
        if highlight:
            extra_spans = highlight.find_all('span')
            extras.append(', '.join([span.get_text(strip=True) for span in extra_spans]))
        else:
            extras.append(' ')

        # Address region from JSON-LD script if necessary
        script = prop.find('script', type='application/ld+json')
        if script:
            data = json.loads(script.string)
            if data.get('@type') == 'Offer':
                available_at_or_from = data.get('availableAtOrFrom', {})
                address_info = available_at_or_from.get('address', {})
                address_region = address_info.get('addressRegion', 'Not provided')
                region.append(address_region)
        else:
            region.append('Not provided')

        sys.stdout.write(f"\rProgress: {int((n / total_pages) * 100)}%")
        sys.stdout.flush()

for i in range(len(title)):
    id.append(i)

sys.stdout.write(f"\rProgress: 100%")
sys.stdout.flush()
print('\nCompleted!')

headers = ['id', 'title', 'price', 'num_rooms', 'total_area', 'latitude', 'longitude', 'region', 'extras']
final_data = [id, title, price, num_rooms, total_area, latitude, longitude, region, extras]
Lisbon_Properties = pd.DataFrame(dict(zip(headers, final_data)))

print(f"\nYou now have data on {len(Lisbon_Properties['id'])} properties located in Lisbon!")

Time estimation for this task: 3 to 5 minutes.
Tasks started at 21h:25min. Estimated finish time 21h:29min.
Progress: 100%
Completed!

You now have data on 1435 properties located in Lisbon!


In [182]:
# Lisbon_Properties.to_csv('Lisbon_Properties.csv')

In [93]:
Lisbon_Properties = pd.read_csv('Lisbon_Properties.csv', index_col=0)
Lisbon_Properties.head()

,id,title,price,num_rooms,total_area,latitude,longitude,region,extras
0,0,"Apartamento T2 em Alvalade, Lisboa",430.000 €,2 quartos,Área bruta 90 m²,"38,7457392","-9,1425898",Alvalade,
1,1,"Apartamento T1 em Benfica, Lisboa",269.900 €,1 quarto,Área bruta 97 m²,"38,75171","-9,2009",Benfica,
2,2,"Apartamento T3 na Rua António Nobre, São Domin...",399.900 €,3 quartos,Área bruta 120 m²,"38,74657","-9,17989",São Domingos de Benfica,
3,3,"Apartamento T2 na Rua de São Pedro Mártir, San...",285.000 €,2 quartos,Área bruta 55 m²,"38,71471","-9,13582",Santa Maria Maior,
4,4,"Apartamento T1 em Praça de Luís de Camões, Mis...",690.000 €,1 quarto,Área bruta 93 m²,"38,71078","-9,14385",Misericórdia,Com garagem


# Lisbon Properties for Sale - Data Preprocessing

## Fixing some issues

### Removing dulicated properties

In [94]:
Lisbon_Properties = Lisbon_Properties[~Lisbon_Properties.drop('id', axis=1).duplicated()]
Lisbon_Properties.reset_index(drop=True, inplace=True)

### Checking Data Quality

In [5]:
Lisbon_Properties['extras'].unique()

array([' ', 'Com garagem', 'De luxo, Com garagem',
       'Com elevador, Com garagem', 'Com elevador',
       'De luxo, Com elevador, Com garagem', 'Rés do chão',
       'De luxo, Piscina, Com elevador',
       'Piscina, Com elevador, Com garagem',
       'Vista para mar, Com elevador, Com garagem',
       'Piscina, Vista para mar, Com garagem', 'Piscina, Com elevador',
       'De luxo, Com elevador', 'Último andar, Com elevador, Com garagem',
       'De luxo, Piscina, Com garagem', 'Piscina, Com garagem',
       'Último andar', 'Rés do chão, Com elevador, Com garagem',
       'De luxo', 'Rés do chão, Com garagem', 'Piscina',
       'De luxo, Vista para mar, Com elevador, Com garagem',
       'Rés do chão, Com elevador',
       'De luxo, Piscina, Rés do chão, Com elevador, Com garagem',
       'De luxo, Rés do chão',
       'De luxo, Piscina, Com elevador, Com garagem',
       'De luxo, Vista para mar, Com garagem',
       'Piscina, Último andar, Com elevador, Com garagem',
       'De 

In [186]:
Lisbon_Properties['latitude'].unique()

array(['38,7457392', '38,75171', '38,74657', '38,71471', '38,71078',
       '38,7234746455', '38,70794', '38,71101', '38,72401', '38,73108',
       '38,71019', '38,72411', '38,70935', '38,70762', '38,72141',
       '38,7321733', '38,70531', '38,7160237', '38,74253', '38,7410926',
       '38,75237', '38,7394789', '38,7116286', '38,7328921', '38,7061891',
       '38,71681', '38,6979033995', '38,71193', '38,72151', '38,77093',
       '38,70501', '38,7185', '38,7506416203', '38,70438', '38,74487',
       '38,7520552', '38,7363924', '38,73121', '38,71006', '38,76273',
       '38,7373789756', '38,76063', '38,7113638', '38,7553', '38,7388',
       '38,74727', '38,70786', '38,71779', '38,71995', '38,74677',
       '38,7146255468', '38,7383787875', '38,738144', '38,7620097658',
       '38,7112', '38,710654', '38,7103093', '38,72876', '38,74445',
       '38,75724', '38,71781', '38,7418702', '38,70627', '38,74049',
       '38,72869', '38,7585032405', '38,71064', '38,710789', '38,74341',
       '3

In [187]:
Lisbon_Properties['longitude'].unique()

array(['-9,1425898', '-9,2009', '-9,17989', '-9,13582', '-9,14385',
       '-9,1581178942', '-9,14468', '-9,14865', '-9,14598', '-9,16051',
       '-9,13348', '-9,13463', '-9,15253', '-9,14239', '-9,13855',
       '-9,1350438', '-9,19939', '-9,1500739', '-9,13526', '-9,1471513',
       '-9,20599', '-9,1520223', '-9,2137529', '-9,1341885', '-9,1616014',
       '-9,12154', '-9,1952884197', '-9,14586', '-9,1595', '-9,09819',
       '-9,16721', '-9,16215', '-9,1987098542', '-9,16618', '-9,17992',
       '-9,2052859', '-9,1451395', '-9,13677', '-9,14999', '-9,17759',
       '-9,09786', '-9,1683963189', '-9,17393', '-9,1476538', '-9,18206',
       '-9,16836', '-9,10117', '-9,14405', '-9,12244', '-9,16478',
       '-9,169', '-9,1642905246', '-9,126318619', '-9,133503',
       '-9,1578005254', '-9,16114', '-9,207782', '-9,1339418096',
       '-9,149471', '-9,17845', '-9,14093', '-9,12247', '-9,158727',
       '-9,16753', '-9,150863', '-9,14937', '-9,0953688697', '-9,1353',
       '-9,1385962',

In [188]:
Lisbon_Properties['price'].unique()

array(['430.000 €', '269.900 €', '399.900 €', '285.000 €', '690.000 €',
       '3.950.000 €', '1.650.000 €', '1.800.000 €', '1.500.000 €',
       '2.200.000 €', '8.200.000 €', '375.000 €', '310.000 €',
       '2.450.000 €', '2.390.000 €', '250.000 €', '325.000 €',
       '320.000 €', '810.000 €', '1.470.000 €', '257.000 €', '495.000 €',
       '488.000 €', '490.000 €', '299.000 €', '1.400.000 €',
       '1.270.000 €', '825.000 €', '1.590.000 €', '758.000 €',
       '1.900.000 €', '615.000 €', '359.000 €', '950.000 €', '279.900 €',
       '339.000 €', '128.000 €', '318.000 €', '980.000 €', '3.750.000 €',
       '349.900 €', '995.000 €', '1.290.000 €', '1.700.000 €',
       '1.080.000 €', '1.495.000 €', '260.000 €', '775.000 €',
       '695.000 €', '1.563.840 €', '1.990.000 €', '1.507.200 €',
       '1.547.520 €', '1.850.000 €', '1.250.000 €', '1.536.000 €',
       '339.500 €', '1.682.000 €', '1.932.000 €', '429.900 €',
       '1.856.000 €', '1.737.000 €', '235.000 €', '560.000 €',
     

In [189]:
Lisbon_Properties['num_rooms'].value_counts()

num_rooms
2 quartos                   445
1 quarto                    320
3 quartos                   281
4 quartos                   149
5 quartos                    28
9 quartos                    12
Área bruta 63 m²              9
6 quartos                     7
7 quartos                     6
Área bruta 46 m²              4
Área bruta 41 m²              4
Área bruta 30 m²              3
Área bruta 60 m²              3
Área útil 63 m²               3
Área bruta 51 m²              3
Área bruta 33 m²              3
8 quartos                     2
C.E.: D                       2
Área bruta 50 m²              2
Área bruta 38 m²              2
Área bruta 39 m²              2
10 quartos                    2
Área bruta 25 m²              2
Área bruta 37 m²              2
C.E.: G                       1
Área útil 45 m²               1
Área bruta 31 m²              1
Área bruta 484 m²             1
Área útil 118 m²              1
Área bruta 55 m²              1
Área bruta 40 m²              

In [190]:
Lisbon_Properties['total_area'].unique()

array(['Área bruta 90 m²', 'Área bruta 97 m²', 'Área bruta 120 m²',
       'Área bruta 55 m²', 'Área bruta 93 m²', 'Área bruta 416 m²',
       'Área útil 233 m²', 'Área bruta 234 m²', 'Área bruta 290 m²',
       'Área bruta 336 m²', 'Área bruta 874 m²', 'Área bruta 60 m²',
       'Área bruta 264 m²', 'Área bruta 262 m²', 'Área útil 88 m²',
       'Área bruta 48 m²', 'Área bruta 68 m²', 'Área bruta 150 m²',
       'Área bruta 228 m²', 'Unknown', 'Área bruta 70 m²',
       'Área bruta 102 m²', 'Área bruta 119 m²', 'Área bruta 230 m²',
       'Área bruta 180 m²', 'Área bruta 134 m²', 'Área bruta 130 m²',
       'Área bruta 138 m²', 'Área bruta 160 m²', 'Área bruta 74 m²',
       'Área bruta 110 m²', 'Área bruta 49 m²', 'Área bruta 302 m²',
       'Área bruta 54 m²', 'C.E.: D', 'Área bruta 85 m²',
       'Área bruta 198 m²', 'Área bruta 76 m²', 'C.E.: A',
       'Área bruta 67 m²', 'Área bruta 168 m²', 'Área bruta 426 m²',
       'Área bruta 44 m²', 'Área bruta 191 m²', 'Área útil 140 m²',

In [191]:
Lisbon_Properties['title'].unique()

array(['Apartamento T2 em Alvalade, Lisboa',
       'Apartamento T1 em Benfica, Lisboa',
       'Apartamento T3 na Rua António Nobre, São Domingos de Benfica, Lisboa',
       'Apartamento T2 na Rua de São Pedro Mártir, Santa Maria Maior, Lisboa',
       'Apartamento T1 em Praça de Luís de Camões, Misericórdia, Lisboa',
       'Apartamento T5 na Rua Sousa Pinto, Santo António, Lisboa',
       'Apartamento T4+1 na Rua de São Paulo, Misericórdia, Lisboa',
       'Apartamento T2+1 em Calçada do Combro, Misericórdia, Lisboa',
       'Apartamento T9 na Rua de Santa Marta, Santo António, Lisboa',
       'Apartamento T5 na Avenida Miguel Torga, Campolide, Lisboa',
       'Moradia T5 na Rua das Pedras Negras, Santa Maria Maior, Lisboa',
       'Apartamento T4 em Arroios, Lisboa',
       'Apartamento T1+1 na Rua da Silva, Misericórdia, Lisboa',
       'Apartamento T3 na Rua do Ferragial, Misericórdia, Lisboa',
       'Moradia T4+1 em Largo do Mastro, Arroios, Lisboa',
       'Apartamento T2+1 em

In [192]:
Lisbon_Properties['region'].value_counts()

region
Estrela                    185
São Domingos de Benfica    142
São Vicente                108
Avenidas Novas             107
Arroios                    103
Santa Maria Maior           80
Misericórdia                68
Campolide                   62
Santo António               54
Benfica                     53
Belém                       49
Alcântara                   44
Parque das Nações           42
Campo de Ourique            41
Penha de França             34
Lumiar                      32
Alvalade                    32
Areeiro                     27
Marvila                     22
Ajuda                       22
Olivais                     10
Carnide                      6
Santa Clara                  4
Beato                        4
Name: count, dtype: int64

### Fixing issues found in previous step

There are some wrong values in 'num_rooms' and 'total_area' features:
1. In 'num_rooms', we have areas, which shouldn't happen2. 
In 'total_area', we have some unexpected values like'C.E.: 

In [95]:
Lisbon_Properties[Lisbon_Properties['num_rooms'].apply(lambda x: 'Área' in x)].head(10)

,id,title,price,num_rooms,total_area,latitude,longitude,region,extras
39,39,Apartamento T0 em Travessa do Terreiro de Sant...,128.000 €,Área bruta 28 m²,C.E.: D,"38,71006","-9,14999",Misericórdia,Rés do chão
134,138,"Apartamento T0 em Travessa do Almargem, Santa ...",850.000 €,Área bruta 48 m²,C.E.: C,"38,7092483961","-9,1316074133",Santa Maria Maior,
172,177,"Apartamento T0 em Campolide, Lisboa",289.000 €,Área bruta 37 m²,C.E.: C,"38,7281213","-9,1624887",Campolide,Último andar
177,182,"Apartamento T0 em Santa Maria Maior, Lisboa",425.000 €,Área bruta 63 m²,C.E.: B-,"38,71064","-9,1353",Santa Maria Maior,
242,253,"Apartamento T0+1 em Avenidas Novas, Lisboa",345.000 €,Área bruta 60 m²,C.E.: B-,"38,7453","-9,15258",Avenidas Novas,"Rés do chão, Com elevador, Com garagem"
255,267,"Apartamento T0 na Rua António Feijó, São Domin...",550.000 €,Área bruta 57 m²,C.E.: C,"38,74559","-9,17853",São Domingos de Benfica,Com elevador
269,283,"Loft T0 Duplex em Misericórdia, Lisboa",299.000 €,Área bruta 60 m²,C.E.: B-,"38,7093","-9,14433",Misericórdia,
273,288,"Apartamento T0 em Alcântara, Lisboa",229.000 €,Área bruta 35 m²,C.E.: E,"38,70362","-9,17891",Alcântara,
286,301,"Apartamento T0 em São Vicente, Lisboa",299.900 €,Área bruta 63 m²,C.E.: D,"38,7177917","-9,1224923",São Vicente,Rés do chão
342,360,"Estúdio T0 na Rua do Vale de Santo António, Sã...",299.900 €,Área bruta 51 m²,C.E.: Propriedade isenta,"38,7178006","-9,1225126",São Vicente,Rés do chão


After filtering the Lisbon_Properties data frame, we found out that these rows were T0s and properties that do not specify the number of rooms.

To fix this, in the next step, we check whether the string 'Área' is in the 'num_rooms' column of each row. 

If this is **True**, we append each index of the row to a list called 'modify_indeces' and:
- If 'T0' is in the title of this specific row, we assign the 'num_rooms' column to be the 'total_area' column, and assign the string '0 rooms' to the 'num_rooms' column;
- Else - for properties that do not speficy the number of rooms - we assign minus 1 ('-1') - the reason of this is that we will fill these missing values with machine learning techniques.

In [96]:
modify_indices = []

for i, rooms in enumerate(Lisbon_Properties['num_rooms']):
    if isinstance(rooms, str) and 'Área' in rooms:
        modify_indices.append(i)

for i in modify_indices:
    if 'T0' in Lisbon_Properties.at[i, 'title']: 
        Lisbon_Properties.at[i, 'total_area'] = Lisbon_Properties.at[i, 'num_rooms']
        Lisbon_Properties.at[i, 'num_rooms'] = '0 quartos'
    else: 
        Lisbon_Properties.at[i, 'total_area'] = Lisbon_Properties.at[i, 'num_rooms']
        Lisbon_Properties.at[i, 'num_rooms'] = '-1'
        
Lisbon_Properties.reset_index(drop=True, inplace=True)

In [97]:
Lisbon_Properties[Lisbon_Properties['title'].apply(lambda x: 'T0' in x)].head(10) # Check whether T0s rows are already fixed

,id,title,price,num_rooms,total_area,latitude,longitude,region,extras
39,39,Apartamento T0 em Travessa do Terreiro de Sant...,128.000 €,0 quartos,Área bruta 28 m²,"38,71006","-9,14999",Misericórdia,Rés do chão
134,138,"Apartamento T0 em Travessa do Almargem, Santa ...",850.000 €,0 quartos,Área bruta 48 m²,"38,7092483961","-9,1316074133",Santa Maria Maior,
172,177,"Apartamento T0 em Campolide, Lisboa",289.000 €,0 quartos,Área bruta 37 m²,"38,7281213","-9,1624887",Campolide,Último andar
177,182,"Apartamento T0 em Santa Maria Maior, Lisboa",425.000 €,0 quartos,Área bruta 63 m²,"38,71064","-9,1353",Santa Maria Maior,
242,253,"Apartamento T0+1 em Avenidas Novas, Lisboa",345.000 €,0 quartos,Área bruta 60 m²,"38,7453","-9,15258",Avenidas Novas,"Rés do chão, Com elevador, Com garagem"
255,267,"Apartamento T0 na Rua António Feijó, São Domin...",550.000 €,0 quartos,Área bruta 57 m²,"38,74559","-9,17853",São Domingos de Benfica,Com elevador
269,283,"Loft T0 Duplex em Misericórdia, Lisboa",299.000 €,0 quartos,Área bruta 60 m²,"38,7093","-9,14433",Misericórdia,
273,288,"Apartamento T0 em Alcântara, Lisboa",229.000 €,0 quartos,Área bruta 35 m²,"38,70362","-9,17891",Alcântara,
286,301,"Apartamento T0 em São Vicente, Lisboa",299.900 €,0 quartos,Área bruta 63 m²,"38,7177917","-9,1224923",São Vicente,Rés do chão
342,360,"Estúdio T0 na Rua do Vale de Santo António, Sã...",299.900 €,0 quartos,Área bruta 51 m²,"38,7178006","-9,1225126",São Vicente,Rés do chão


After these corrections, we will drop the properties that do not have either the correct number of rooms or the correct total area.

In [98]:
drop_indices = []

for i,area in enumerate(Lisbon_Properties['total_area']):
    if isinstance(area, str) and 'Área' not in area:
        drop_indices.append(i)

Lisbon_Properties.drop(index=drop_indices, inplace=True)

### Checking if our fixes went well

In [99]:
Lisbon_Properties['num_rooms'].value_counts()

num_rooms
2 quartos     439
1 quarto      317
3 quartos     280
4 quartos     147
0 quartos      47
5 quartos      28
-1             24
9 quartos      12
6 quartos       7
7 quartos       6
10 quartos      2
8 quartos       2
11 quartos      1
Name: count, dtype: int64

In [100]:
Lisbon_Properties['total_area'].unique() # Checking if our fixes went well

array(['Área bruta 90 m²', 'Área bruta 97 m²', 'Área bruta 120 m²',
       'Área bruta 55 m²', 'Área bruta 93 m²', 'Área bruta 416 m²',
       'Área útil 233 m²', 'Área bruta 234 m²', 'Área bruta 290 m²',
       'Área bruta 336 m²', 'Área bruta 874 m²', 'Área bruta 60 m²',
       'Área bruta 264 m²', 'Área bruta 262 m²', 'Área útil 88 m²',
       'Área bruta 48 m²', 'Área bruta 68 m²', 'Área bruta 150 m²',
       'Área bruta 228 m²', 'Área bruta 70 m²', 'Área bruta 102 m²',
       'Área bruta 119 m²', 'Área bruta 230 m²', 'Área bruta 180 m²',
       'Área bruta 134 m²', 'Área bruta 130 m²', 'Área bruta 138 m²',
       'Área bruta 160 m²', 'Área bruta 74 m²', 'Área bruta 110 m²',
       'Área bruta 49 m²', 'Área bruta 302 m²', 'Área bruta 54 m²',
       'Área bruta 28 m²', 'Área bruta 85 m²', 'Área bruta 198 m²',
       'Área bruta 76 m²', 'Área bruta 67 m²', 'Área bruta 168 m²',
       'Área bruta 426 m²', 'Área bruta 44 m²', 'Área bruta 191 m²',
       'Área útil 140 m²', 'Área útil 1

## Modifying and Creating New Features

In [101]:
Lisbon_Properties.head()

,id,title,price,num_rooms,total_area,latitude,longitude,region,extras
0,0,"Apartamento T2 em Alvalade, Lisboa",430.000 €,2 quartos,Área bruta 90 m²,"38,7457392","-9,1425898",Alvalade,
1,1,"Apartamento T1 em Benfica, Lisboa",269.900 €,1 quarto,Área bruta 97 m²,"38,75171","-9,2009",Benfica,
2,2,"Apartamento T3 na Rua António Nobre, São Domin...",399.900 €,3 quartos,Área bruta 120 m²,"38,74657","-9,17989",São Domingos de Benfica,
3,3,"Apartamento T2 na Rua de São Pedro Mártir, San...",285.000 €,2 quartos,Área bruta 55 m²,"38,71471","-9,13582",Santa Maria Maior,
4,4,"Apartamento T1 em Praça de Luís de Camões, Mis...",690.000 €,1 quarto,Área bruta 93 m²,"38,71078","-9,14385",Misericórdia,Com garagem


### Defining some functions to modify some columns

In [102]:
def extra_rooms(x):
    if '+' in x:
        return int(x.split('+')[1])
    else:
        return 0

def area(x):
    if 'Área' in x:
        return int(x.split()[2].replace('.',''))
    else:
        return 'Unknown'

def num_extras(x):
    if x.isspace():
        return 0
    elif ',' in x:
        return int(len(x.split(',')))
    else:
        return 1

In [103]:
Lisbon_Properties['Type'] = Lisbon_Properties['title'].apply(lambda x: x.split()[0])
Lisbon_Properties['Typology'] = Lisbon_Properties['title'].apply(lambda x: x.split()[1])
Lisbon_Properties['Extra_Rooms'] = Lisbon_Properties['Typology'].apply(extra_rooms)
Lisbon_Properties['Extra_Rooms_Flag'] = Lisbon_Properties['Extra_Rooms'].apply(lambda x: 1 if x > 0 else 0)
Lisbon_Properties['N_Rooms'] = Lisbon_Properties['num_rooms'].apply(lambda x: int(x.split()[0]))
Lisbon_Properties['Total_N_Rooms'] = Lisbon_Properties['N_Rooms'] + Lisbon_Properties['Extra_Rooms']
Lisbon_Properties['Price'] = Lisbon_Properties['price'].apply(lambda x: int(x.replace('.','').split()[0]))
Lisbon_Properties['Area_m2'] = Lisbon_Properties['total_area'].apply(area)
Lisbon_Properties['N_Extras'] = Lisbon_Properties['extras'].apply(num_extras)
Lisbon_Properties['Extras_Flag'] = Lisbon_Properties['N_Extras'].apply(lambda x: 1 if x > 0 else 0)
Lisbon_Properties['Latitude'] = Lisbon_Properties['latitude'].apply(lambda x: float(x.replace(',','.')))
Lisbon_Properties['Longitude'] = Lisbon_Properties['longitude'].apply(lambda x: float(x.replace(',','.')))
Lisbon_Properties.rename(columns={'region':'Region'}, inplace=True)

**Summary:**

- Modified Columns:
  1. N_rooms - Number of property rooms (e.g. 3 integer, instead of '3 rooms' string)
  2. Price - Property price in euros (integer instead of string - e.g. 300.000 € --> Price = 300000)
  3. Area_m2 = Property area in square meters (m2) (e.g. 'Área bruta 90 m²' --> Area_m2 = 90)
  4. Latitude = Property latitude (e.g. '38,7457392' string --> Latitude = 38,7457392 float)
  5. Longitude = Property latitude (e.g. '-9,1425898' string --> Latitude = -9,1425898 float)
 
- New Columns:
  1. Type - Property type (e.g. Apartment, Duplex, etc.)
  2. Typology - Property typology (e.g. T1, T2, etc.) - this is just a temporary column to extract extra rooms
  3. Extra_Rooms - Property extra rooms (e.g. T1+1 --> Extra_Rooms = 1)
  4. Extra_Rooms_Flag - 1 if the property has extra rooms, 0 otherwise
  5. N_Extras - Number of property extras
  6. Extras_Flag - 1 if the property has extras, 0 otherwise
  7. Total_N_Rooms - Total number of property rooms (number of rooms + extra rooms - e.g. T1+1 --> Total_N_Rooms = 2)

### Finding the unique extras to create dummy variables for it

In [104]:
extras_df = Lisbon_Properties['extras'].str.split(',', expand=True)

extras = []
unique_extras = []

for x in [0,1,2,3,4]:
    extras.append(list(extras_df[x].unique()))

for ext in extras:
    unique_extras += ext

unique_extras = set(unique_extras)
unique_extras = {extra.strip() for extra in unique_extras if extra is not None and extra.strip()}

In [105]:
unique_extras

{'Com elevador',
 'Com garagem',
 'De luxo',
 'Piscina',
 'Rés do chão',
 'Vista para mar',
 'Último andar'}

In [106]:
for feature in unique_extras:
    Lisbon_Properties[feature] = Lisbon_Properties['extras'].apply(lambda x: int(feature in x))

### Creating dummy variables for Region and Property Type

In [107]:
region_dummies = pd.get_dummies(Lisbon_Properties['Region'],dtype=int)
type_dummies = pd.get_dummies(Lisbon_Properties['Type'],dtype=int)

In [108]:
Lisbon_Properties = pd.concat([Lisbon_Properties, region_dummies, type_dummies], axis=1)

### Drop Columns that we no longer need

In [109]:
columns_to_drop = ['title', 'price', 'num_rooms', 'total_area', 'latitude', 'longitude', 'Typology','extras','Region','Type']
Lisbon_Properties = Lisbon_Properties.drop(columns_to_drop, axis=1)

In [110]:
Lisbon_Properties.head()

,id,Extra_Rooms,Extra_Rooms_Flag,N_Rooms,Total_N_Rooms,Price,Area_m2,N_Extras,Extras_Flag,Latitude,...,Apartamento,Casa,Duplex,Estúdio,Flat,Loft,Moradia,Palacete,Penthouse,Quinta
0,0,0,0,2,2,430000,90,0,0,38.745739,...,1,0,0,0,0,0,0,0,0,0
1,1,0,0,1,1,269900,97,0,0,38.751710,...,1,0,0,0,0,0,0,0,0,0
2,2,0,0,3,3,399900,120,0,0,38.746570,...,1,0,0,0,0,0,0,0,0,0
3,3,0,0,2,2,285000,55,0,0,38.714710,...,1,0,0,0,0,0,0,0,0,0
4,4,0,0,1,1,690000,93,1,1,38.710780,...,1,0,0,0,0,0,0,0,0,0


In [22]:
Lisbon_Properties.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1312 entries, 0 to 1330
Data columns (total 52 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       1312 non-null   int64  
 1   Extra_Rooms              1312 non-null   int64  
 2   Extra_Rooms_Flag         1312 non-null   int64  
 3   N_Rooms                  1312 non-null   int64  
 4   Total_N_Rooms            1312 non-null   int64  
 5   Price                    1312 non-null   int64  
 6   Area_m2                  1312 non-null   int64  
 7   N_Extras                 1312 non-null   int64  
 8   Extras_Flag              1312 non-null   int64  
 9   Latitude                 1312 non-null   float64
 10  Longitude                1312 non-null   float64
 11  Último andar             1312 non-null   int64  
 12  Piscina                  1312 non-null   int64  
 13  Rés do chão              1312 non-null   int64  
 14  De luxo                  1312

# Lisbon Metro Info - Wikipedia Webscrapping

In [172]:
url = 'https://pt.wikipedia.org/wiki/Lista_de_esta%C3%A7%C3%B5es_do_Metropolitano_de_Lisboa'
response = requests.get(url)
soup = bs4.BeautifulSoup(response.text, 'html.parser')
data = str(soup.find('table', {'class': 'wikitable'}))

table = pd.read_html(data)[0]
columns = ['Nome','Outros nomes','Linha','Lat.','Long.']
Lisbon_Metro = table[columns]

In [175]:
# Lisbon_Metro.to_csv('Lisbon_Metro.csv')

In [25]:
Lisbon_Metro = pd.read_csv('Lisbon_Metro.csv', index_col=0)
Lisbon_Metro

,Nome,Outros nomes,Linha,Lat.,Long.
0,Aeroporto,—,Vermelha,38.76861,−9.12861
1,Alameda,Alameda I (técn.),Verde,38.73713,−9.13388
2,Alameda,Alameda II (técn.),Vermelha,38.73697,−9.13261
3,Alfornelos,—,Azul,38.76038,−9.20435
4,Alto dos Moinhos,Centro Administrativo (prev.),Azul,38.74994,−9.18003
5,Alvalade,—,Verde,38.75311,−9.14396
6,Amadora Este,Falagueira (prev.),Azul,38.75847,−9.21803
7,Ameixoeira,Carriche (prev.),Amarela,38.77937,−9.15962
8,Anjos,—,Verde,38.72715,−9.13485
9,Areeiro,—,Verde,38.74233,−9.13354


# Lisbon Metro Info - Data Preprocessing

## Removing last row that contains the same info as the heather of the data frame

In [26]:
Lisbon_Metro.drop(index=56, inplace=True)

In [27]:
Lisbon_Metro

,Nome,Outros nomes,Linha,Lat.,Long.
0,Aeroporto,—,Vermelha,38.76861,−9.12861
1,Alameda,Alameda I (técn.),Verde,38.73713,−9.13388
2,Alameda,Alameda II (técn.),Vermelha,38.73697,−9.13261
3,Alfornelos,—,Azul,38.76038,−9.20435
4,Alto dos Moinhos,Centro Administrativo (prev.),Azul,38.74994,−9.18003
5,Alvalade,—,Verde,38.75311,−9.14396
6,Amadora Este,Falagueira (prev.),Azul,38.75847,−9.21803
7,Ameixoeira,Carriche (prev.),Amarela,38.77937,−9.15962
8,Anjos,—,Verde,38.72715,−9.13485
9,Areeiro,—,Verde,38.74233,−9.13354


## Creating a new column - Nome_Concat (Concats the name and the sub-name, aka 'Outros nomes', of each Lisbon Metro Station)

In [28]:
Lisbon_Metro['Nome_Concat'] = Lisbon_Metro['Nome']+' - '+Lisbon_Metro['Outros nomes']
Lisbon_Metro.head()

,Nome,Outros nomes,Linha,Lat.,Long.,Nome_Concat
0,Aeroporto,—,Vermelha,38.76861,−9.12861,Aeroporto - —
1,Alameda,Alameda I (técn.),Verde,38.73713,−9.13388,Alameda - Alameda I (técn.)
2,Alameda,Alameda II (técn.),Vermelha,38.73697,−9.13261,Alameda - Alameda II (técn.)
3,Alfornelos,—,Azul,38.76038,−9.20435,Alfornelos - —
4,Alto dos Moinhos,Centro Administrativo (prev.),Azul,38.74994,−9.18003,Alto dos Moinhos - Centro Administrativo (prev.)


In [91]:
Lisbon_Metro['Lat.'] = Lisbon_Metro['Lat.'].apply(lambda x: float(x))
Lisbon_Metro['Long.'] = Lisbon_Metro['Long.'].apply(lambda x: x.replace('−','-'))
Lisbon_Metro['Long.'] = Lisbon_Metro['Long.'].apply(lambda x: float(x))

### Note: before we convert 'Long.' to float, we had to replace the symbol '−' by a real minus '-'

# Lisbon Cultural Facilities Info - Lisboa Aberta Webscrapping: Create Cultural Score

## Museaums Data

In [217]:
museums_url = 'https://services.arcgis.com/1dSrzEWVQn5kHHyK/arcgis/rest/services/POICultura/FeatureServer/3/query?where=1%3D1&outFields=*&f=pgeojson'

response = requests.get(museums_url)
response.raise_for_status()

mdf = gpd.read_file(response.text)

Lisbon_Museums = pd.DataFrame(mdf)
Lisbon_Museums.head()

,OBJECTID,COD_SIG,INF_NOME,INF_MORADA,FREGUESIA,INF_TELEFONE,INF_EMAIL,INF_SITE,INF_DESCRICAO,INF_ACTIVO,GlobalID,geometry
0,1,656,Museu de Arte Popular,Avenida de Brasília,Belém,+ 351 213 011 282,geral@map.dgpc.pt,https://museuartepopular.wordpress.com/,"Inaugurado em 1948, o seu acervo é essencialme...",1,4188e21c-373e-429e-b209-95c8ec28a34c,POINT (-9.20845 38.69380)
1,3,658,Museu Coleção Berardo,Praça do Império\nCentro Cultural de Belém\n\n,Belém,+351 213 612 878,museuberardo@museuberardo.pt,www.museuberardo.pt,O Museu Coleção Berardo abriu portas a 25 de j...,1,e9b18708-da7a-4c91-87b4-dc1a4646a55b,POINT (-9.20773 38.69575)
2,4,659,"MAAT - Museu de Arte, Arquitetura e Tecnologia",Avenida de Brasília,Belém,+351 210 028 130,info.maat@edp.pt,https://www.maat.pt,"O MAAT, propriedade da Fundação EDP, é um espa...",1,d30df7a4-89fd-4570-a5e2-aca9fdff84f8,POINT (-9.19372 38.69604)
3,5,660,Museu Nacional dos Coches,Avenida da Índia,Belém,+ 351 210 732 319,geral@mncoches.dgpc.pt,http://museudoscoches.gov.pt/pt/,"O atual edifício do Museu dos Coches, desenhad...",1,be5d44a3-dd01-4258-a3ae-c3b73e74e446,POINT (-9.19839 38.69680)
4,6,661,Museu de Marinha,Praça do Império \nMosteiro dos Jerónimos,Belém,+351 210 977 988,geral.museu@marinha.pt,http://ccm.marinha.pt/pt/museu,"Organizado em zonas temáticas distintas, poder...",1,658ca5c9-bc5b-49c7-ba5b-18659c32036b,POINT (-9.20924 38.69723)


### Save a backup

In [318]:
# Lisbon_Museums = Lisbon_Museums.to_csv('Lisbon_Museums.csv')

In [122]:
Lisbon_Museums = pd.read_csv('Lisbon_Museums.csv', index_col=0)

## Theaters Data

In [219]:
theaters_url = 'https://services.arcgis.com/1dSrzEWVQn5kHHyK/arcgis/rest/services/POICultura/FeatureServer/4/query?where=1%3D1&outFields=*&f=pgeojson'

response = requests.get(theaters_url)
response.raise_for_status()

tdf = gpd.read_file(response.text)

Lisbon_Theaters = pd.DataFrame(tdf)
Lisbon_Theaters.head()

,OBJECTID,COD_SIG,INF_NOME,INF_MORADA,FREGUESIA,INF_TELEFONE,INF_EMAIL,INF_SITE,INF_DESCRICAO,INF_ACTIVO,GlobalID,geometry
0,1,758,Gabinete curiosidades Karnart,"Avenida da Índia, 168",Belém,+351 213 466 411/+351 914 150 935,geral@karnart.org,www.karnart.org,O Gabinete Curiosidades Karnart é a sede da KA...,1,3fe9f7da-7fad-4b14-acb3-52f2e9c0c4a6,POINT (-9.21166 38.69427)
1,2,759,Academia Dramática Familiar 1º de Novembro,"Rua da Praia de Pedrouços, 76-78",Belém,+351 213 012 601,None,None,Edifício do séc XIX de arquitectura cultural e...,1,54f50c25-86c4-4cfa-afda-56bc0df13bde,POINT (-9.21781 38.69461)
2,5,762,LU.CA - Teatro Luís de Camões,"Calçada da Ajuda, 80",Belém,+351 215 939 100,info@lucateatroluisdecamoes.pt,www.lucateatroluisdecamoes.pt,"Pequeno teatro de gosto neoclássico tardio, in...",1,bfdcb6c3-99a0-4295-a270-91707f347131,POINT (-9.19938 38.69903)
3,6,763,Espaço POGO,"Rua da Cintura do Porto de Lisboa, Edifício 403",Estrela,+351 916 403 994,info@pogo.pt,http://www.pogo.pt,None,1,51727d32-af42-4cae-bc34-9628b47b8d51,POINT (-9.15388 38.70570)
4,7,764,Teatro Cinearte,"Largo de Santos, 2-2E",Estrela,21 396 53 60/275,barraca@mail.telepac.pt / bilheteira@abarraca.com,www.abarraca.com,Encomendado pela Sociedade Administradora de C...,1,70165c4b-3536-4eb2-83bd-36d07a1110e8,POINT (-9.15524 38.70718)


### Save a backup

In [321]:
# Lisbon_Theaters.to_csv('Lisbon_Theaters.csv')

In [123]:
Lisbon_Theaters = pd.read_csv('Lisbon_Theaters.csv', index_col=0)

## Cinemas Data

In [223]:
cin_url = 'https://services.arcgis.com/1dSrzEWVQn5kHHyK/arcgis/rest/services/POICultura/FeatureServer/2/query?where=1%3D1&outFields=*&f=pgeojson'

response = requests.get(cin_url)
response.raise_for_status() 

cdf = gpd.read_file(response.text)

Lisbon_Cinemas = pd.DataFrame(cdf)
Lisbon_Cinemas.head()

,OBJECTID,COD_SIG,INF_NOME,INF_MORADA,FREGUESIA,INF_TELEFONE,INF_EMAIL,INF_SITE,INF_DESCRICAO,INF_FONTE,INF_ACTIVO,GlobalID,geometry
0,1,506,Cinema Ideal,"Rua do Loreto, 15",Misericórdia,+351 210 998 295,cinemaideal@cinemaideal.pt,www.cinemaideal.pt,"Esta sala de cinema, aberta desde 1904, conhec...",-,1,45afe625-02d7-4712-b6b8-7aa0c2116ed6,POINT (-9.14422 38.71054)
1,2,507,Cinemateca Júnior,"Praça dos Restauradores, Palácio da Foz",Santa Maria Maior,+351 213 462 157 | +351 213 476 129,cinemateca.junior@cinemateca.pt,http://www.cinemateca.pt/,"A Cinemateca Júnior, instalada no Palácio Foz,...",http://www.cinemateca.pt/,1,1d3ca135-5a24-478c-95f4-dd2dc87a65c9,POINT (-9.14217 38.71556)
2,3,508,Cinema São Jorge,"Avenida da Liberdade, 175",Santo António,+351 213 103 400,cinemasaojorge@egeac.pt,http://cinemasaojorge.pt/,"Um dos mais emblemáticos cinemas de Lisboa, co...",http://cinemasaojorge.pt/,1,1bf79faf-5d97-48b3-b085-fa4446c435a2,POINT (-9.14633 38.72029)
3,4,509,Cinemateca Portuguesa,"Rua Barata Salgueiro, 39 R/C",Santo António,+ 351 213 596 200,cinemateca@cinemateca.pt,www.cinemateca.pt/,"Em 1948 foi criada a Cinemateca Nacional, hoje...",-,1,3414bf94-62ac-43c7-a846-0d5e41251e89,POINT (-9.14875 38.72088)
4,5,510,NOS Amoreiras,Avenida Engenheiro Duarte Pacheco Lote 7 2º Lo...,Campo de Ourique,16996,cinema.amoreiras@nos.pt,http://cinemas.nos.pt/,Possui 7 salas\n,http://cinemas.nos.pt/,1,1af99162-1626-4f83-9342-f891e2b442bc,POINT (-9.16210 38.72367)


### Save a backup

In [325]:
# Lisbon_Cinemas.to_csv('Lisbon_Cinemas.csv')

In [124]:
Lisbon_Cinemas = pd.read_csv('Lisbon_Cinemas.csv', index_col=0)

## Auditoriums Data

In [227]:
aud_url = 'https://services.arcgis.com/1dSrzEWVQn5kHHyK/arcgis/rest/services/POICultura/FeatureServer/1/query?where=1%3D1&outFields=*&f=pgeojson'

response = requests.get(aud_url)
response.raise_for_status()

adf = gpd.read_file(response.text)

Lisbon_Auditoriums = pd.DataFrame(adf)
Lisbon_Auditoriums.head()

,OBJECTID,COD_SIG,INF_NOME,INF_MORADA,FREGUESIA,INF_TELEFONE,INF_EMAIL,INF_SITE,INF_DESCRICAO,INF_FONTE,INF_ACTIVO,GlobalID,geometry
0,1,168,Centro de Documentação da Casa da América Latina,"Avenida da Índia, 110",Belém,+351 218 172 490,geral@casamericalatina.pt,http://casamericalatina.pt/inicio/,None,-,1,3d123d45-0d43-4507-b6b3-37a4f384a251,POINT (-9.19434 38.69672)
1,2,169,UCCLA - Centro de Documentação,"Avenida da Índia, 110",Belém,+351 218 172 950,uccla@uccla.pt,http://www.uccla.pt/,A UCCLA é uma associação intermunicipal de nat...,-,1,d3cc88bf-d350-4f9d-829a-94cd08ffebb8,POINT (-9.19434 38.69675)
2,3,170,Biblioteca de Belém,Rua da Junqueira 295,Belém,+351 218 172 580,bib.belem@cm-lisboa.pt,http://blx.cm-lisboa.pt,A Biblioteca Municipal de Belém encontra-se in...,CML - Rede BLx,1,53473127-6007-4d47-9b87-fa4e563c56f8,POINT (-9.19584 38.69713)
3,4,171,Arquivo Central e Arquivo Histórico de Marinha,Avenida da Índia S/N 1º,Belém,21 362 76 00,arquivo.central@marinha.pt / arquivo.historico...,www.marinha.pt,Fundos com origem na documentação da antiga Se...,www.marinha.pt,1,ae67622f-e6cc-4a72-afd3-f69f2d4039cb,POINT (-9.19237 38.69717)
4,5,172,Biblioteca Central da Marinha,Praça do Império Museu da Marinha R/C,Belém,21 365 85 20,biblioteca.marinha@marinha.pt,www.marinha.pt,"Situada, desde 1891, na Praça do Império, na i...",www.marinha.pt,1,fd047b92-5e5f-45f4-bf47-c703cfb0012f,POINT (-9.20924 38.69720)


### Save a backup

In [328]:
# Lisbon_Auditoriums.to_csv('Lisbon_Auditoriums.csv')

In [125]:
Lisbon_Auditoriums = pd.read_csv('Lisbon_Auditoriums.csv', index_col=0)

## Creating a static column 'Type' to identify the facilities of each rows (e.g. Museums, Theaters, etc)

In [126]:
Lisbon_Museums['Type'] = 'Museum'
Lisbon_Theaters['Type'] = 'Theater'
Lisbon_Cinemas['Type'] = 'Cinema'
Lisbon_Auditoriums['Type'] = 'Auditorium'

# Cultural Dataframe

In [141]:
culture_list = [Lisbon_Museums, Lisbon_Theaters, Lisbon_Cinemas, Lisbon_Auditoriums] 
Lisbon_Culture = pd.concat(culture_list)
Lisbon_Culture.head()

,OBJECTID,COD_SIG,INF_NOME,INF_MORADA,FREGUESIA,INF_TELEFONE,INF_EMAIL,INF_SITE,INF_DESCRICAO,INF_ACTIVO,GlobalID,geometry,Type,INF_FONTE
0,1,656,Museu de Arte Popular,Avenida de Brasília,Belém,+ 351 213 011 282,geral@map.dgpc.pt,https://museuartepopular.wordpress.com/,"Inaugurado em 1948, o seu acervo é essencialme...",1,4188e21c-373e-429e-b209-95c8ec28a34c,POINT (-9.20844625414179 38.6938036001826),Museum,NaN
1,3,658,Museu Coleção Berardo,Praça do Império\nCentro Cultural de Belém\n\n,Belém,+351 213 612 878,museuberardo@museuberardo.pt,www.museuberardo.pt,O Museu Coleção Berardo abriu portas a 25 de j...,1,e9b18708-da7a-4c91-87b4-dc1a4646a55b,POINT (-9.20773166084022 38.6957476923844),Museum,NaN
2,4,659,"MAAT - Museu de Arte, Arquitetura e Tecnologia",Avenida de Brasília,Belém,+351 210 028 130,info.maat@edp.pt,https://www.maat.pt,"O MAAT, propriedade da Fundação EDP, é um espa...",1,d30df7a4-89fd-4570-a5e2-aca9fdff84f8,POINT (-9.19371529028373 38.6960435304729),Museum,NaN
3,5,660,Museu Nacional dos Coches,Avenida da Índia,Belém,+ 351 210 732 319,geral@mncoches.dgpc.pt,http://museudoscoches.gov.pt/pt/,"O atual edifício do Museu dos Coches, desenhad...",1,be5d44a3-dd01-4258-a3ae-c3b73e74e446,POINT (-9.19838680167877 38.6968006044798),Museum,NaN
4,6,661,Museu de Marinha,Praça do Império \nMosteiro dos Jerónimos,Belém,+351 210 977 988,geral.museu@marinha.pt,http://ccm.marinha.pt/pt/museu,"Organizado em zonas temáticas distintas, poder...",1,658ca5c9-bc5b-49c7-ba5b-18659c32036b,POINT (-9.20923637432389 38.6972269914078),Museum,NaN


# Cultural Dataframe - Data Preprocessing

### Transform the 'geometry' column to Shapely Point objects to subsequently extract latitude and longitude

In [142]:
Lisbon_Culture['geometry'] = Lisbon_Culture['geometry'].apply(wkt.loads)

In [143]:
Lisbon_Culture.dtypes

OBJECTID          int64
COD_SIG           int64
INF_NOME         object
INF_MORADA       object
FREGUESIA        object
INF_TELEFONE     object
INF_EMAIL        object
INF_SITE         object
INF_DESCRICAO    object
INF_ACTIVO        int64
GlobalID         object
geometry         object
Type             object
INF_FONTE        object
dtype: object

## Drop columns that we no longer need

In [144]:
colms_to_drop = ['COD_SIG', 'INF_MORADA', 'INF_TELEFONE', 'INF_EMAIL', 'INF_SITE', 'INF_DESCRICAO', 'INF_ACTIVO', 'INF_FONTE']
Lisbon_Culture.drop(colms_to_drop, axis=1, inplace=True)

## Checking duplicates

In [145]:
duplicates = Lisbon_Culture.duplicated()
print("Duplicate rows:")
print(Lisbon_Culture[duplicates])

Duplicate rows:
Empty DataFrame
Columns: [OBJECTID, INF_NOME, FREGUESIA, GlobalID, geometry, Type]
Index: []


In [146]:
duplicates = Lisbon_Culture.duplicated(subset=['GlobalID', 'geometry'])
print("Duplicate rows based on columns 'GlobalID' and 'geometry':")
print(Lisbon_Culture[duplicates])

Duplicate rows based on columns 'GlobalID' and 'geometry':
Empty DataFrame
Columns: [OBJECTID, INF_NOME, FREGUESIA, GlobalID, geometry, Type]
Index: []


There are no duplicates

## Spliting Geometry Into Lat. Long. Coordinates

### Defining a function for this task

In [147]:
def extract_lat_long(point):
    if isinstance(point, Point):
        latitude = point.y
        longitude = point.x
        return pd.Series([latitude, longitude])
    else:
        return pd.Series([None, None])

In [148]:
Lisbon_Culture[['latitude', 'longitude']] = Lisbon_Culture['geometry'].apply(extract_lat_long)

In [149]:
Lisbon_Culture.head()

,OBJECTID,INF_NOME,FREGUESIA,GlobalID,geometry,Type,latitude,longitude
0,1,Museu de Arte Popular,Belém,4188e21c-373e-429e-b209-95c8ec28a34c,POINT (-9.20844625414179 38.6938036001826),Museum,38.693804,-9.208446
1,3,Museu Coleção Berardo,Belém,e9b18708-da7a-4c91-87b4-dc1a4646a55b,POINT (-9.20773166084022 38.6957476923844),Museum,38.695748,-9.207732
2,4,"MAAT - Museu de Arte, Arquitetura e Tecnologia",Belém,d30df7a4-89fd-4570-a5e2-aca9fdff84f8,POINT (-9.19371529028373 38.6960435304729),Museum,38.696044,-9.193715
3,5,Museu Nacional dos Coches,Belém,be5d44a3-dd01-4258-a3ae-c3b73e74e446,POINT (-9.19838680167877 38.6968006044798),Museum,38.696801,-9.198387
4,6,Museu de Marinha,Belém,658ca5c9-bc5b-49c7-ba5b-18659c32036b,POINT (-9.20923637432389 38.6972269914078),Museum,38.697227,-9.209236


### Drop 'geometry' column since it is no longer needed

In [150]:
Lisbon_Culture.drop('geometry', axis=1, inplace=True)

### Saving a backup

In [274]:
# Lisbon_Culture.to_csv('Lisbon_Culture.csv')

# Lisbon Educational Facilities Info - Lisboa Aberta Webscrapping - Create Educational Facilities Score

## Pre-School (Public only)

In [33]:
pre_public_url = 'https://services.arcgis.com/1dSrzEWVQn5kHHyK/arcgis/rest/services/POIEducacao/FeatureServer/9/query?outFields=*&where=1%3D1&f=geojson'

response = requests.get(pre_public_url)
response.raise_for_status()

pre_pub = gpd.read_file(response.text)

Lisbon_PrePublic = pd.DataFrame(pre_pub)

### Save a backup

In [34]:
Lisbon_PrePublic.to_csv('Lisbon_PrePublic.csv')

In [35]:
Lisbon_PrePublic = pd.read_csv('Lisbon_PrePublic.csv', index_col=0)

### Creating 'Acess' and 'Stage' columns to identify rows as Pre-School and Public

In [36]:
Lisbon_PrePublic['Access'] = 'Public'
Lisbon_PrePublic['Stage'] = 'Pre-School'

### Rename Columns to Concat

In [37]:
Lisbon_PrePublic.rename(columns={'INF_NOME': 'Name', 'INF_MORADA': 'Address', 'INF_EMAIL': 'Email', 'INF_SITE': 'Website','INF_ACTIVO':'Flag_is_Active', 'FREGUESIA': 'Area'}, inplace=True)

### Drop useless columns

In [38]:
columns_to_drop = ['OBJECTID', 'COD_SIG', 'Address', 'INF_TELEFONE', 'Email', 'Website', 'Flag_is_Active', 'INF_DESCRICAO', 'INF_ID']
Lisbon_PrePublic.drop(columns_to_drop, axis=1, inplace=True)

### Creating new columns order

In [39]:
new_order = ['Name', 'Area', 'GlobalID','geometry', 'Access', 'Stage']
Lisbon_PrePublic = Lisbon_PrePublic[new_order]

Lisbon_PrePublic.head()

,Name,Area,GlobalID,geometry,Access,Stage
0,Escola Profissional Metropolitana de Lisboa,Alcântara,707e37d5-e1b5-45fc-85f8-cc7095308431,POINT (-9.18158997715356 38.699297532228),Public,Pre-School
1,Conservatório Metropolitano de Música de Lisboa,Alcântara,83edfbca-6558-460e-93d1-0475467960a1,POINT (-9.1813018524282 38.6993993202234),Public,Pre-School
2,Academia de Música de Luisboa - Os Violinos,Belém,a421d20f-eb13-4126-a30c-2158b07fe2fd,POINT (-9.1944372593028 38.7001317342682),Public,Pre-School
3,Acordarte - Academia de Música de Lisboa,Belém,24aa8435-1660-4cdd-a993-5113daefd230,POINT (-9.1944372593028 38.7001317342682),Public,Pre-School
4,Escola Profissional de Imagem (ETIC),Misericórdia,049de07d-4405-4a0f-8946-4ddce0c71d28,POINT (-9.14745713248909 38.7078031160822),Public,Pre-School


## 1st Cicle Public/Private

In [40]:
# Public Schools
first_public_url = 'https://services.arcgis.com/1dSrzEWVQn5kHHyK/arcgis/rest/services/POIEducacao/FeatureServer/1/query?outFields=*&where=1%3D1&f=geojson'

response = requests.get(first_public_url)
response.raise_for_status()

Lisbon_1stPublic = gpd.read_file(response.text)
Lisbon_1stPublic = pd.DataFrame(Lisbon_1stPublic)

# Private Schools
first_private_url = 'https://services.arcgis.com/1dSrzEWVQn5kHHyK/arcgis/rest/services/POIEducacao/FeatureServer/6/query?where=1%3D1&outFields=*&f=pgeojson'

response = requests.get(first_private_url)
response.raise_for_status()

Lisbon_1stPrivate  = gpd.read_file(response.text)
Lisbon_1stPrivate = pd.DataFrame(Lisbon_1stPrivate)

### Save a backup

In [41]:
# Lisbon_1stPublic.to_csv('Lisbon_1stPublic.csv')
# Lisbon_1stPrivate.to_csv('Lisbon_1stPrivate.csv')

In [42]:
Lisbon_1stPublic = pd.read_csv('Lisbon_1stPublic.csv',index_col=0)
Lisbon_1stPrivate = pd.read_csv('Lisbon_1stPrivate.csv',index_col=0)

### Creating 'Acess' and 'Stage' columns to identify rows as 1st Cicle and Public/Private

In [43]:
Lisbon_1stPublic['Access'] = 'Public'
Lisbon_1stPublic['Stage'] = 'First'
Lisbon_1stPrivate['Access'] = 'Private'
Lisbon_1stPrivate['Stage'] = 'First'

### Rename Columns to Concat

In [44]:
Lisbon_1stPublic.rename(columns={'NOME_ESCOLA': 'Name', 'MORADA': 'Address', 'FREGUESIA12': 'Area'}, inplace=True)
Lisbon_1stPrivate.rename(columns={'INF_NOME': 'Name', 'INF_MORADA': 'Address', 'FREGUESIA': 'Area', 'INF_SITE': 'Website'}, inplace=True)

### Drop useless columns by creating subsets of needed data

In [45]:
needed_columns = ['Name', 'Area', 'GlobalID', 'geometry', 'Access', 'Stage']

Lisbon_1stPublic = Lisbon_1stPublic[needed_columns]
Lisbon_1stPrivate = Lisbon_1stPrivate[needed_columns]

### Creating new columns order

In [46]:
new_order = ['Name', 'Area', 'GlobalID','geometry', 'Access', 'Stage']

Lisbon_1stPublic = Lisbon_1stPublic[new_order]
Lisbon_1stPrivate = Lisbon_1stPrivate[new_order]

## 2nd/3rd Cicle Public/Private

In [47]:
# Public
second_public_url = 'https://services.arcgis.com/1dSrzEWVQn5kHHyK/arcgis/rest/services/POIEducacao/FeatureServer/1/query?outFields=*&where=1%3D1&f=geojson'

response = requests.get(second_public_url)
response.raise_for_status() 

Lisbon_2ndPublic  = gpd.read_file(response.text)
Lisbon_2ndPublic = pd.DataFrame(Lisbon_2ndPublic)

# Private
second_private_url = 'https://services.arcgis.com/1dSrzEWVQn5kHHyK/arcgis/rest/services/POIEducacao/FeatureServer/6/query?outFields=*&where=1%3D1&f=geojson'

response = requests.get(second_private_url)
response.raise_for_status() 

Lisbon_2ndPrivate  = gpd.read_file(response.text)
Lisbon_2ndPrivate = pd.DataFrame(Lisbon_2ndPrivate)

### Save a backup

In [48]:
# Lisbon_2ndPublic.to_csv('Lisbon_2ndPublic.csv')
# Lisbon_2ndPrivate.to_csv('Lisbon_2ndPrivate.csv')

In [49]:
Lisbon_2ndPublic = pd.read_csv('Lisbon_2ndPublic.csv', index_col=0)
Lisbon_2ndPrivate = pd.read_csv('Lisbon_2ndPrivate.csv', index_col=0)

### Creating 'Acess' and 'Stage' columns to identify rows as 2nd/3d Cicle and Public/Private

In [50]:
Lisbon_2ndPublic['Access'] = 'Public'
Lisbon_2ndPublic['Stage'] = 'Second'

Lisbon_2ndPrivate['Access'] = 'Private'
Lisbon_2ndPrivate['Stage'] = 'Second'

### Rename Columns to Concat


In [51]:
Lisbon_2ndPublic.rename(columns={'NOME_ESCOLA': 'Name', 'MORADA': 'Address', 'FREGUESIA12': 'Area', 'INF_SITE': 'Website'}, inplace=True)
Lisbon_2ndPrivate.rename(columns={'INF_NOME': 'Name', 'INF_MORADA': 'Address', 'FREGUESIA': 'Area', 'SITE': 'Website'}, inplace=True)

### Drop useless columns by creating subsets of needed data

In [55]:
needed_columns = ['Name', 'Area', 'GlobalID', 'geometry', 'Access', 'Stage']

Lisbon_2ndPublic = Lisbon_1stPublic[needed_columns]
Lisbon_2ndPrivate = Lisbon_1stPrivate[needed_columns]

### Creating new columns order

In [56]:
new_order = ['Name', 'Area', 'GlobalID','geometry', 'Access', 'Stage']

Lisbon_2ndPublic = Lisbon_2ndPublic[new_order]
Lisbon_2ndPrivate = Lisbon_2ndPrivate[new_order]

## High School Public/Private

In [57]:
# Public
high_public_url = 'https://services.arcgis.com/1dSrzEWVQn5kHHyK/arcgis/rest/services/POIEducacao/FeatureServer/1/query?outFields=*&where=1%3D1&f=geojson'

response = requests.get(high_public_url)
response.raise_for_status()

Lisbon_HSchool_Public = gpd.read_file(response.text)
Lisbon_HSchool_Public = pd.DataFrame(Lisbon_HSchool_Public)

#Private
high_private_url = 'https://services.arcgis.com/1dSrzEWVQn5kHHyK/arcgis/rest/services/POIEducacao/FeatureServer/7/query?outFields=*&where=1%3D1&f=geojson'

response = requests.get(high_private_url)
response.raise_for_status()

Lisbon_HSchool_Private  = gpd.read_file(response.text)
Lisbon_HSchool_Private = pd.DataFrame(Lisbon_HSchool_Private)

### Save a backup

In [58]:
Lisbon_HSchool_Public.to_csv('Lisbon_HSchool_Public.csv')
Lisbon_HSchool_Private.to_csv('Lisbon_HSchool_Private.csv')

### Creating 'Acess' and 'Stage' columns to identify rows as High School and Public/Private

In [59]:
Lisbon_HSchool_Public['Access'] = 'Public'
Lisbon_HSchool_Public['Stage'] = 'Second'

Lisbon_HSchool_Private['Access'] = 'Private'
Lisbon_HSchool_Private['Stage'] = 'HighSchool'

### Rename Columns to Concat

In [60]:
Lisbon_HSchool_Public.rename(columns={'INF_NOME': 'Name', 'INF_MORADA': 'Address', 'FREGUESIA': 'Area', 'INF_SITE': 'Website'}, inplace=True)
Lisbon_HSchool_Private.rename(columns={'INF_NOME': 'Name', 'INF_MORADA': 'Address', 'FREGUESIA': 'Area', 'INF_SITE': 'Website'}, inplace=True)

### Drop useless columns by creating subsets of needed data

In [61]:
needed_columns = ['Name', 'Area', 'GlobalID', 'geometry', 'Access', 'Stage']

Lisbon_HSchool_Public = Lisbon_1stPublic[needed_columns]
Lisbon_HSchool_Private = Lisbon_1stPrivate[needed_columns]

### Creating new columns order

In [62]:
new_order = ['Name', 'Area', 'GlobalID','geometry', 'Access', 'Stage']
Lisbon_HSchool_Public = Lisbon_HSchool_Public[new_order]
Lisbon_HSchool_Private = Lisbon_HSchool_Private[new_order]

## College Public/Private

In [63]:
uni_url = 'https://services.arcgis.com/1dSrzEWVQn5kHHyK/arcgis/rest/services/MapaConhecimento/FeatureServer/1/query?where=1%3D1&outFields=*&f=pgeojson'

response = requests.get(uni_url)
response.raise_for_status() 

uni_all  = response.json()

Lisbon_AllUni = gpd.GeoDataFrame.from_features(uni_all['features'])

Lisbon_AllUni = pd.DataFrame(Lisbon_AllUni)

### Save a backup

In [64]:
# Lisbon_AllUni.to_csv('Lisbon_AllUni.csv')

In [65]:
Lisbon_AllUni = pd.read_csv('Lisbon_AllUni.csv', index_col=0)

### Creating 'Acess' and 'Stage' columns to identify rows as Universities and All types

In [66]:
Lisbon_AllUni['Access'] = 'All'
Lisbon_AllUni['Stage'] = 'Uni'

### Rename Columns to Concat

In [67]:
Lisbon_AllUni.rename(columns={'NOME': 'Name', 'MORADA': 'Address', 'SITE': 'Website'}, inplace=True)
Lisbon_AllUni['Area'] = 'NaN'

### Drop useless columns by creating subsets of needed data

In [68]:
columns_to_drop_uni = ['OBJECTID', 'COD_SIG', 'IDTIPO', 'Address', 'ATORTIPOCODIGO', 'EMAIL', 'Website', 'REDESOCIAL', 'REDESOCIAL']
Lisbon_AllUni.drop(columns_to_drop_uni, axis=1, inplace=True)

### Creating new columns order

In [69]:
needed_columns = ['Name', 'Area', 'GlobalID', 'geometry', 'Access', 'Stage']
Lisbon_AllUni = Lisbon_AllUni[new_order]

# Educational Dataframe

## Check Common Columns for Concat

In [71]:
edu_list = [Lisbon_PrePublic, Lisbon_1stPrivate, Lisbon_1stPublic, Lisbon_2ndPrivate, Lisbon_2ndPublic, Lisbon_HSchool_Public, Lisbon_HSchool_Private, Lisbon_AllUni]

common_columns = set(edu_list[0].columns)
for df in edu_list[1:]:
    common_columns.intersection_update(df.columns)

print(f"Common columns: {common_columns}")

Common columns: {'Name', 'Access', 'geometry', 'Area', 'GlobalID', 'Stage'}


## Concat Dataframes

In [72]:
Lisbon_Education = pd.concat(edu_list)
Lisbon_Education

,Name,Area,GlobalID,geometry,Access,Stage
0,Escola Profissional Metropolitana de Lisboa,Alcântara,707e37d5-e1b5-45fc-85f8-cc7095308431,POINT (-9.18158997715356 38.699297532228),Public,Pre-School
1,Conservatório Metropolitano de Música de Lisboa,Alcântara,83edfbca-6558-460e-93d1-0475467960a1,POINT (-9.1813018524282 38.6993993202234),Public,Pre-School
2,Academia de Música de Luisboa - Os Violinos,Belém,a421d20f-eb13-4126-a30c-2158b07fe2fd,POINT (-9.1944372593028 38.7001317342682),Public,Pre-School
3,Acordarte - Academia de Música de Lisboa,Belém,24aa8435-1660-4cdd-a993-5113daefd230,POINT (-9.1944372593028 38.7001317342682),Public,Pre-School
4,Escola Profissional de Imagem (ETIC),Misericórdia,049de07d-4405-4a0f-8946-4ddce0c71d28,POINT (-9.14745713248909 38.7078031160822),Public,Pre-School
...,...,...,...,...,...,...
80,ue | Universidade Europeia - FCSD (Faculdade d...,NaN,bd396f8b-c726-4df0-aa96-b6714df4c562,POINT (-9.18365790945365 38.7650161641021),All,Uni
81,ue | Universidade Europeia - FCES (Faculdade d...,NaN,fd0bce17-613a-4c5a-96a6-b55b17478aa9,POINT (-9.19368269569772 38.7600490654187),All,Uni
82,UNL | NOVA IMS - NOVA Information Management S...,NaN,4515c8c1-81ec-4ba6-be95-79b575a776c7,POINT (-9.16030254570311 38.7325791296909),All,Uni
83,UNL | ITQB NOVA - Instituto de Tecnologia Quím...,NaN,23ef0db1-af6f-4549-a281-1e9196115849,POINT (-9.32163033090969 38.6959016232198),All,Uni


### Save a backup

In [73]:
# Lisbon_Education.to_csv('Lisbon_Education.csv')

In [74]:
Lisbon_Education = pd.read_csv('Lisbon_Education.csv', index_col=0)

# Educational Dataframe - Data Preprocessing

## Check Duplicates

In [75]:
duplicates = Lisbon_Education.duplicated(subset=['GlobalID', 'geometry'])
print("Duplicate rows based on columns 'GlobalID' and 'geometry':")
print(Lisbon_Education[duplicates])

Duplicate rows based on columns 'GlobalID' and 'geometry':
                                Name                     Area  \
0             Colégio do Bom Sucesso                    Belém   
1            Externato Alfredo Binet                    Belém   
2              Cooperativa "A Torre"                    Belém   
3            Externato "A Escolinha"                    Belém   
4   Externato "Santa Maria de Belém"                    Belém   
..                               ...                      ...   
94                 Colégio Place4all         Campo de Ourique   
95           Escola Casa da Floresta  São Domingos de Benfica   
96              Colégio "O Pelicano"                  Areeiro   
97                   Colégio Oriente        Parque das Nações   
98           Externato Alfredo Binet                    Belém   

                                GlobalID  \
0   15cf71ac-1d1e-46d7-a0e7-7f09140d06c2   
1   d693914b-1dc3-42a8-9a06-379bfaf954e2   
2   91d9dfd8-af85-4f8e-a585-

## Drop duplicates

In [76]:
Lisbon_Education = Lisbon_Education.drop_duplicates(subset=['GlobalID', 'geometry'])

## Check nulls

In [77]:
Lisbon_Education['geometry'].isnull().sum()

2

## Drop nulls

In [78]:
Lisbon_Education = Lisbon_Education.dropna(subset=['geometry']).reset_index(drop=True)

In [79]:
Lisbon_Education['geometry'].isnull().sum()

0

## Transform the 'geometry' column to Shapely Point objects to subsequently extract latitude and longitude

In [80]:
Lisbon_Education['geometry'] = Lisbon_Education['geometry'].apply(wkt.loads)

In [84]:
Lisbon_Education[['latitude', 'longitude']] = Lisbon_Education['geometry'].apply(extract_lat_long)

In [86]:
Lisbon_Education.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Name       300 non-null    object 
 1   Area       215 non-null    object 
 2   GlobalID   300 non-null    object 
 3   geometry   300 non-null    object 
 4   Access     300 non-null    object 
 5   Stage      300 non-null    object 
 6   latitude   300 non-null    float64
 7   longitude  300 non-null    float64
dtypes: float64(2), object(6)
memory usage: 18.9+ KB


# Me

## Function to calculate distances with latitude and longitude features

<html>
<p><strong>Great Circle Distance Formula:</strong></p>
<p>The formula to calculate the great circle or 'as the crow flies' distance between two points on the Earth's surface, given their latitude and longitude is:</p>
<p style="font-family: 'Lucida Console', Monaco, monospace;">
  \( d = 2R \times \sin^{-1}\left(\sqrt{\sin^2\left(\frac{\theta_2 - \theta_1}{2}\right) + \cos \theta_1 \times \cos \theta_2 \times \sin^2\left(\frac{\phi_2 - \phi_1}{2}\right)}\right) \)
</p>
<p>where:</p>
<ul>
  <li><strong>\( (\theta_1, \phi_1) \)</strong> and <strong>\( (\theta_2, \phi_2) \)</strong> – Coordinates of each point (latitude and longitude, respectively);</li>
  <li><strong>R</strong> – Radius of the Earth; and</li>
  <li><strong>d</strong> – Great circle distance between the points.</li>
</l>
</html>


In [87]:
def haversine(lat1, lon1, lat2, lon2):
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)
    
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    R = 6371.0
    distance = R * c
    
    return distance

### We decided to define a threshold of 1.5 km

In [88]:
dist_threshold = 1.5

### Calculate Distances between each Property and each Metro Station and add them to Lisbon_Properties Dataframe

In [111]:
distances = []

for p_index, p_row in Lisbon_Properties.iterrows():
    count_nearby_stations = 0
    for m_index, m_row in Lisbon_Metro.iterrows():
        distance = haversine(p_row['Latitude'], p_row['Longitude'], m_row['Lat.'], m_row['Long.'])
        if distance < dist_threshold:
            count_nearby_stations += 1
        distances.append({
            'property_id': p_row['id'],
            #'property_title': p_row['title'],
            'metro_station': m_row['Nome'],
            'distance_km': distance
        })
    Lisbon_Properties.at[p_index, 'Stations_within_1.5km'] = count_nearby_stations

In [112]:
Distances_df = pd.DataFrame(distances)

In [113]:
Distances_df.head()

,property_id,metro_station,distance_km
0,0.0,Aeroporto,2.817243
1,0.0,Alameda,1.219446
2,0.0,Alameda,1.303829
3,0.0,Alfornelos,5.597536
4,0.0,Alto dos Moinhos,3.280310


### Convert 'property_id' to integer

In [114]:
Distances_df['property_id'] = Distances_df['property_id'].apply(lambda x: int(x))

### Example: Distances of property_id = 1 to each metro station

In [115]:
Distances_df.loc[Distances_df['property_id'] == 0]

,property_id,metro_station,distance_km
0,0,Aeroporto,2.817243
1,0,Alameda,1.219446
2,0,Alameda,1.303829
3,0,Alfornelos,5.597536
4,0,Alto dos Moinhos,3.280310
5,0,Alvalade,0.828164
6,0,Amadora Este,6.693329
7,0,Ameixoeira,4.020537
8,0,Anjos,2.173306
9,0,Areeiro,0.871611


In [118]:
Lisbon_Properties.head()

,id,Extra_Rooms,Extra_Rooms_Flag,N_Rooms,Total_N_Rooms,Price,Area_m2,N_Extras,Extras_Flag,Latitude,...,Casa,Duplex,Estúdio,Flat,Loft,Moradia,Palacete,Penthouse,Quinta,stations_within_1.5km
0,0,0,0,2,2,430000,90,0,0,38.745739,...,0,0,0,0,0,0,0,0,0,9.0
1,1,0,0,1,1,269900,97,0,0,38.751710,...,0,0,0,0,0,0,0,0,0,4.0
2,2,0,0,3,3,399900,120,0,0,38.746570,...,0,0,0,0,0,0,0,0,0,4.0
3,3,0,0,2,2,285000,55,0,0,38.714710,...,0,0,0,0,0,0,0,0,0,11.0
4,4,0,0,1,1,690000,93,1,1,38.710780,...,0,0,0,0,0,0,0,0,0,9.0


### Converting Stations_within_1.5km to integer

In [119]:
Lisbon_Properties['Stations_within_1.5km'] = Lisbon_Properties['Stations_within_1.5km'].apply(lambda x: int(x))

## Calculate the number of cultural facilities in a 1.5Km radius

In [151]:
dist_threshold_2 = 1.5

cult_distances = []

for p_index, p_row in Lisbon_Properties.iterrows():
    count_nearby_cult = 0
    for c_index, c_row in Lisbon_Culture.iterrows():
        distance = haversine(p_row['Latitude'], p_row['Longitude'], c_row['latitude'], c_row['longitude'])
        if distance < dist_threshold_2:
            count_nearby_cult += 1
        cult_distances.append({
            'property_id': p_row['id'],
            'Name': c_row['INF_NOME'],
            'Area': c_row['FREGUESIA'],
            'distance_km': distance
        })
    Lisbon_Properties.at[p_index, 'Cultural_fac_within_1.5km'] = count_nearby_cult

Cult_Distances_df = pd.DataFrame(cult_distances)

Cult_Distances_df.head()

,property_id,Name,Area,distance_km
0,0.0,Museu de Arte Popular,Belém,8.123650
1,0.0,Museu Coleção Berardo,Belém,7.927054
2,0.0,"MAAT - Museu de Arte, Arquitetura e Tecnologia",Belém,7.085770
3,0.0,Museu Nacional dos Coches,Belém,7.283127
4,0.0,Museu de Marinha,Belém,7.907485


In [153]:
Lisbon_Properties.head()

,id,Extra_Rooms,Extra_Rooms_Flag,N_Rooms,Total_N_Rooms,Price,Area_m2,N_Extras,Extras_Flag,Latitude,...,Duplex,Estúdio,Flat,Loft,Moradia,Palacete,Penthouse,Quinta,stations_within_1.5km,cultural_fac_within_1.5km
0,0,0,0,2,2,430000,90,0,0,38.745739,...,0,0,0,0,0,0,0,0,9,42.0
1,1,0,0,1,1,269900,97,0,0,38.751710,...,0,0,0,0,0,0,0,0,4,8.0
2,2,0,0,3,3,399900,120,0,0,38.746570,...,0,0,0,0,0,0,0,0,4,16.0
3,3,0,0,2,2,285000,55,0,0,38.714710,...,0,0,0,0,0,0,0,0,11,96.0
4,4,0,0,1,1,690000,93,1,1,38.710780,...,0,0,0,0,0,0,0,0,9,106.0


## Converting Cultural_fac_within_1.5km to integer

In [154]:
Lisbon_Properties['Cultural_fac_within_1.5km'] = Lisbon_Properties['Cultural_fac_within_1.5km'].apply(lambda x: int(x))

In [156]:
dist_threshold_3 = 1.5

edu_distances = []

for p_index, p_row in Lisbon_Properties.iterrows():
    count_nearby_edu = 0
    for m_index, m_row in Lisbon_Education.iterrows():
        distance = haversine(p_row['Latitude'], p_row['Longitude'], m_row['latitude'], m_row['longitude'])
        if distance < dist_threshold_3:
            count_nearby_edu += 1
        edu_distances.append({
            'property_id': p_row['id'],
            'Name': m_row['Name'],
            'Area': m_row['Area'],
            'distance_km': distance
        })
    Lisbon_Properties.at[p_index, 'Edu_within_1.5km'] = count_nearby_edu

Edu_distances_df = pd.DataFrame(edu_distances)

Edu_distances_df

,property_id,Name,Area,distance_km
0,0.0,Escola Profissional Metropolitana de Lisboa,Alcântara,6.173723
1,0.0,Conservatório Metropolitano de Música de Lisboa,Alcântara,6.150574
2,0.0,Academia de Música de Luisboa - Os Violinos,Belém,6.778580
3,0.0,Acordarte - Academia de Música de Lisboa,Belém,6.778580
4,0.0,Escola Profissional de Imagem (ETIC),Misericórdia,4.239379
...,...,...,...,...
393595,1436.0,ue | Universidade Europeia - FCSD (Faculdade d...,NaN,5.229345
393596,1436.0,ue | Universidade Europeia - FCES (Faculdade d...,NaN,5.590179
393597,1436.0,UNL | NOVA IMS - NOVA Information Management S...,NaN,1.846459
393598,1436.0,UNL | ITQB NOVA - Instituto de Tecnologia Quím...,NaN,16.380897


## Converting Edu_within_1.5km to integer

In [157]:
Lisbon_Properties['Edu_within_1.5km'] = Lisbon_Properties['Edu_within_1.5km'].apply(lambda x: int(x))

In [159]:
Lisbon_Properties.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1312 entries, 0 to 1330
Data columns (total 55 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         1312 non-null   int64  
 1   Extra_Rooms                1312 non-null   int64  
 2   Extra_Rooms_Flag           1312 non-null   int64  
 3   N_Rooms                    1312 non-null   int64  
 4   Total_N_Rooms              1312 non-null   int64  
 5   Price                      1312 non-null   int64  
 6   Area_m2                    1312 non-null   int64  
 7   N_Extras                   1312 non-null   int64  
 8   Extras_Flag                1312 non-null   int64  
 9   Latitude                   1312 non-null   float64
 10  Longitude                  1312 non-null   float64
 11  Último andar               1312 non-null   int64  
 12  Piscina                    1312 non-null   int64  
 13  Rés do chão                1312 non-null   int64  
 1

# Liner Regression Model

Since we will now apply a linear regression model in order to predict prices for properties based on their features, we can now exclude Latitude and Longitude of each property

In [161]:
Lisbon_Properties.drop(['Latitude','Longitude'], axis=1, inplace=True)